In [2]:
import cv2
import mediapipe as mp
import math

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Webcam feed
cap = cv2.VideoCapture(0)

# Dictionary to map finger names to landmark indices
finger_points = {
    "Thumb": 4,
    "Index": 8,
    "Middle": 12,
    "Ring": 16,
    "Little": 20
}

# Gesture thresholds (you can adjust these based on your needs).
gesture_thresholds = {
    "Stop": 0.03,    # Adjust according to your hand size and distance from camera
    "Rock": 0.03,    # Adjust according to your hand size and distance from camera
    "Hello": 0.03,   # Adjust according to your hand size and distance from camera
    "Bye": 0.03,     # Adjust according to your hand size and distance from camera
    "Hi": 0.03,     # Adjust according to your hand size and distance from camera
    "Hai": 0.03,     # Adjust according to your hand size and distance from camera
    "Peace": 0.03,     # Adjust according to your hand size and distance from camera
    "Scissor": 0.03,     # Adjust according to your hand size and distance from camera
    "Pointing": 0.03,     # Adjust according to your hand size and distance from camera
    "Okay": 0.03,     # Adjust according to your hand size and distance from camera
    "Call Me": 0.03,     # Adjust according to your hand size and distance from camera
    "Super": 0.03,     # Adjust according to your hand size and distance from camera
    "One": 0.03,     # Adjust according to your hand size and distance from camera
    "Two": 0.03,     # Adjust according to your hand size and distance from camera
    "Three": 0.03,     # Adjust according to your hand size and distance from camera
    "Four": 0.03,     # Adjust according to your hand size and distance from camera
}

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Flip the image horizontally for a later selfie-view display
    image = cv2.flip(image, 1)

    # Convert the BGR image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image_rgb)

    # Draw hand landmarks and detect gestures
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Check for finger counting
            finger_count = 0
            for finger_name, point in finger_points.items():
                # Get finger landmark points
                landmark = hand_landmarks.landmark[point]

                # Draw a circle at each finger tip
                tip_x = int(landmark.x * image.shape[1])
                tip_y = int(landmark.y * image.shape[0])
                cv2.circle(image, (tip_x, tip_y), 8, (0, 255, 0), thickness=-1)  # Blue circles

                # Get the base point index
                base_point = point -1
                base_landmark = hand_landmarks.landmark[base_point]

                base_x = int(base_landmark.x * image.shape[1])
                base_y = int(base_landmark.y * image.shape[0])

                # Check if finger is extended (tip above base)
                if tip_y < base_y:
                    finger_count += 1

            # Check for specific gestures like "Stop", "Rock", "Hello", and "Bye"
            thumb_tip = hand_landmarks.landmark[4]
            index_tip = hand_landmarks.landmark[8]
            middle_tip = hand_landmarks.landmark[12]
            ring_tip = hand_landmarks.landmark[16]
            little_tip = hand_landmarks.landmark[20]

            # Calculate distances between specific finger tips for gestures
            distance_stop = (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2
            distance_three = (thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2
            distance_rock = (thumb_tip.x - middle_tip.x)**2 + (thumb_tip.y - middle_tip.y)**2
            distance_hello = (thumb_tip.x - ring_tip.x)**2 + (thumb_tip.y - ring_tip.y)**2
            # distance_callMe = (thumb_tip.x - little_tip.x)**2 + (thumb_tip.y - little_tip.y)**2
            distance_hai = (index_tip.x - middle_tip.x)**2 + (index_tip.y - middle_tip.y)**2
            distance_hi = (index_tip.x - middle_tip.x)**2 + (index_tip.y - middle_tip.y)**2
            distance_pointing = (index_tip.x - thumb_tip.x)**2 + (index_tip.y - thumb_tip.y)**2
            distance_peace = (index_tip.x - ring_tip.x)**2 + (index_tip.y - ring_tip.y)**2
            distance_callMe = (index_tip.x - ring_tip.x)**2 + (index_tip.y - ring_tip.y)**2
            distance_super = (middle_tip.x - ring_tip.x)**2 + (middle_tip.y - ring_tip.y)**2
            distance_scissor = (middle_tip.x - index_tip.x)**2 + (middle_tip.y - index_tip.y)**2
            distance_okay= (ring_tip.x - middle_tip.x)**2 + (ring_tip.y - middle_tip.y)**2
            distance_bye = (ring_tip.x - little_tip.x)**2 + (ring_tip.y - little_tip.y)**2
            distance_two = (ring_tip.x - index_tip.x)**2 + (ring_tip.y - index_tip.y)**2
            distance_one = (little_tip.x - index_tip.x)**2 + (middle_tip.y - index_tip.y)**2
            distance_four = (little_tip.x - thumb_tip.x)**2 + (middle_tip.y - thumb_tip.y)**2

            # Check if distances meet gesture thresholds
            if distance_stop < gesture_thresholds["Stop"] and finger_count == 5:
                cv2.putText(image, "Gesture: Stop", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif distance_rock < gesture_thresholds["Rock"] and finger_count == 0:
                cv2.putText(image, "Gesture: Rock", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif distance_bye < gesture_thresholds["Bye"] and finger_count == 5:
                cv2.putText(image, "Gesture: Bye", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif distance_hai < gesture_thresholds["Hai"] and finger_count == 5:
                cv2.putText(image, "Gesture: Hai", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif distance_hello < gesture_thresholds["Hello"] and finger_count == 3:
                cv2.putText(image, "Gesture: Hello", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif distance_super < gesture_thresholds["Super"] and finger_count == 3:
                cv2.putText(image, "Gesture: Super", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif finger_count == 3:
                cv2.putText(image, "Gesture: Three", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif distance_scissor < gesture_thresholds["Scissor"] and finger_count == 2:
                cv2.putText(image, "Gesture: Scissor", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif finger_count == 2:
                cv2.putText(image, "Gesture: Two", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif distance_hi < gesture_thresholds["Hi"] and finger_count == 2:
                cv2.putText(image, "Gesture: Hi", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif distance_callMe < gesture_thresholds["Call Me"] and finger_count == 2:
                cv2.putText(image, "Gesture: Call Me", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif distance_peace < gesture_thresholds["Peace"] and finger_count == 2:
                cv2.putText(image, "Gesture: Peace", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif distance_pointing < gesture_thresholds["Pointing"] and finger_count == 1:
                cv2.putText(image, "Gesture: Pointing", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif distance_okay < gesture_thresholds["Okay"] and finger_count == 1:
                cv2.putText(image, "Gesture: Okay", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif finger_count == 1:
                cv2.putText(image, "Gesture: One", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif finger_count == 4:
                cv2.putText(image, "Gesture: Four", (350, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Display finger count
            cv2.putText(image, f"Fingers: {finger_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show the image
    cv2.imshow('Hand Gesture Recognition', image)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()